# The London Railway Network

The cell below defines the abstract class whose API you will need to impement. Do NOT modify it.

In [6]:
# DO NOT MODIFY THIS CELL

from abc import ABC, abstractmethod  

class AbstractLondonRailwayMapper(ABC):
    
    # constructor
    @abstractmethod
    def __init__(self):
        pass           
        
    # data initialisation
    @abstractmethod
    def loadStationsAndLines(self):
        pass

    # returns the minimum number of stops to connect station "fromS" to station  "toS"
    # fromS : str
    # toS : str
    # numStops : int
    @abstractmethod
    def minStops(self, fromS, toS):     
        numStops = -1
        return numStops    
    
    # returns the minimum distance in miles to connect station "fromS" to station  "toS"
    # fromS : str
    # toS : str
    # minDistance : float
    @abstractmethod
    def minDistance(self, fromS, toS):
        minDistance = -1.0
        return minDistance
    
    # given an unordered list of station names, returns a new railway line 
    # (represented as a list of adjacent station names), connecting all such stations 
    # and such that the sum of the distances (in miles) between adjacent stations is minimised
    # inputList : set<str>
    # outputList : list<str>
    @abstractmethod
    def newRailwayLine(self, inputList):
        outputList = []
        return outputList

Use the cell below to define any data structure and auxiliary python function you may need. Leave the implementation of the main API to the next code cell instead.

In [7]:
import math

# ADD YOUR DATA STRUCTURE DEFINITIONS AND HELPER CODE HERE
class Station:
    def __init__(self, name, latitude, longitude):
        self.name = name
        self.latitude = latitude
        self.longitude = longitude
        self.relation = {}
        self.line = []

    def getDistance(self, station):
        earthRadius = 6371.004 #km average radius
        latself = float(self.latitude)
        latstation = float(station.latitude)
        lonself = float(self.longitude)
        lonstation = float(station.longitude)
        C = math.sin(latstation)*math.sin(latself) + math.cos(latstation)*math.cos(latself)*math.cos(lonstation - lonself)
        
        return earthRadius * math.acos(C)*math.pi/180.0

    def addRelation(self, station):
        if station.name not in self.relation.keys():
            distance = self.getDistance(station)
            self.relation[station.name] = distance
            station.relation[self.name] = distance

    def addLineinfo(self, linename):
        if linename not in self.line:
            self.line.append(linename)
    
class Network:
    def __init__(self):
        self.stations = {}
    
    def addStation(self, name, latitude, longitude):
        if name not in self.stations.keys():
            station = Station(name, latitude, longitude)
            self.stations[name] = station
    
    def addLine(self, stationA, stationB, linename):
        stationA.addRelation(stationB)
        stationA.addLineinfo(linename)
        stationB.addLineinfo(linename)

    def getStation(self, name):
        return self.stations.get(name)

    



Use the cell below to implement the requested API.

In [8]:
import csv
import random

class LondonRailwayMapper(AbstractLondonRailwayMapper):

    def __init__(self):
        # ADD YOUR CODE HERE
        self.railwayNetwork = self.loadStationsAndLines()
        
    def loadStationsAndLines(self):
        # ADD YOUR CODE HERE
        railwayNetwork = Network()
        csvFilestations = open("londonstations.csv", "r")
        csvFilerailwaylines = open("londonrailwaylines.csv", "r")
        reader0 = csv.reader(csvFilestations)
        reader1 = csv.reader(csvFilerailwaylines)
        csvFilestations.readline()
        csvFilerailwaylines.readline()
        for row in reader0:
            railwayNetwork.addStation(row[0], row[1], row[2])
        csvFilestations.close()

        for row in reader1:
            stationA = railwayNetwork.getStation(row[1])
            stationB = railwayNetwork.getStation(row[2])
            railwayNetwork.addLine(stationA, stationB, row[0])
        
        return railwayNetwork
        
        
    
    

    def minStops(self, fromS, toS):     
        numStops = -1
        # ADD YOUR CODE HERE
        toStation = self.railwayNetwork.getStation(toS)
        queue = {}
        queue[fromS] = 0
        seen = set()
        seen.add(fromS)
        parent = {fromS:None}
        minimum = {}
        previous = fromS
        for station in self.railwayNetwork.stations.keys():
            minimum[station] = 114514
        minimum[fromS] = 0
        while len(queue) > 0:
            queue = dict(sorted(queue.items(), key = lambda kv:(kv[1], kv[0]), reverse=True))
            vertex = queue.popitem()
            vertexName = vertex[0]
            nodes = self.railwayNetwork.getStation(vertexName).relation.keys()
            if vertexName == toS:
                return minimum.get(vertexName)
            if vertexName == fromS:
                stops = minimum.get(previous)
            else:
                stops = minimum.get(parent.get(vertexName)[0]) + 1

            if minimum[vertexName] > stops:
                minimum[vertexName] = stops
            if toS in nodes:
                return minimum.get(vertexName) + 1
            for station in nodes:
                if station not in seen:
                    score = stops #self.railwayNetwork.getStation(station).getDistance(toStation) + stops
                    queue[station] = score
                    seen.add(station)
                    parent[station] = vertex
                if toS in self.railwayNetwork.getStation(station).relation.keys():
                    return minimum.get(parent.get(station)[0]) + 2
            previous = vertexName
            
    
        return numStops    
    
    
    
    def minDistance(self, fromS, toS):
        minDistance = -1.0
        # ADD YOUR CODE HERE
        toStation = self.railwayNetwork.getStation(toS)
        queue = {}
        queue[fromS] = 0
        seen = set()
        seen.add(fromS)
        parent = {fromS:None}
        minimum = {}
        previous = fromS
        for station in self.railwayNetwork.stations.keys():
            minimum[station] = 1145141919810
        minimum[fromS] = 0
        while len(queue) > 0:
            queue = dict(sorted(queue.items(), key = lambda kv:(kv[1], kv[0]), reverse=True))
            vertex = queue.popitem()
            vertexName = vertex[0]
            nodes = self.railwayNetwork.getStation(vertexName).relation.keys()
            if vertexName == fromS:
                stops = minimum.get(previous)
            else:
                stops = minimum.get(parent.get(vertexName)[0]) + self.railwayNetwork.getStation(parent.get(vertexName)[0]).getDistance(self.railwayNetwork.getStation(vertexName))

            if minimum[vertexName] > stops:
                minimum[vertexName] = stops
            if toS in nodes:
                return minimum.get(vertexName)*0.62137
            for station in nodes:
                if station not in seen:
                    score = self.railwayNetwork.getStation(station).getDistance(toStation) + stops
                    queue[station] = score
                    seen.add(station)
                    parent[station] = vertex
            previous = vertexName
        
        return minDistance
        
    
    
    def newRailwayLine(self, inputList):
        outputList = []
        # ADD YOUR CODE HERE
        numberset = []
        distset = []
        temperatureset = []
        #inuse.reverse()
        temperature = 10000
        decreaseFactor = 0.995
        numrange = 500000
        mindis = 1145141919810
        #plt.figure(figsize = (20,20))
        for j in range(1):
            inuse = inputList.copy()
            random.shuffle(inuse)
            distanceA = 0
            samecounter = 0
            for i in range(len(inuse)):
                if i < len(inuse) - 1:
                    distanceA += self.railwayNetwork.getStation(inuse[i]).getDistance(self.railwayNetwork.getStation(inuse[i+1]))
            for num in range(numrange):
                changelist = inuse.copy()
                countA = 0
                countB = 0
                distanceB = 0
                while countA == countB:
                    countA = random.randint(0,len(inuse)-1)
                    countB = random.randint(0,len(inuse)-1)
                if countA > countB:
                    temp = changelist[countB]
                    changelist[countB] = changelist[countA]
                    changelist[countA] = temp
                else:
                    temp = changelist[countA]
                    changelist[countA] = changelist[countB]
                    changelist[countB] = temp
                for i in range(len(changelist)):
                    if i < len(changelist) - 1:
                        distanceB += self.railwayNetwork.getStation(changelist[i]).getDistance(self.railwayNetwork.getStation(changelist[i+1]))
                if distanceA > distanceB:
                    inuse = changelist.copy()
                    distanceA = distanceB
                    samecounter = 0
                    
                else:
                    p = math.exp((distanceA - distanceB)/temperature)
                    r = random.random()
                    if p > r:
                        inuse = changelist.copy()
                        distanceA = distanceB
                        samecounter = 0                        
                    else:
                        samecounter += 1
                        if samecounter >= 1000:
                            if mindis > distanceA:
                                mindis = distanceA
                                outputList = inuse.copy()
                                outputList.append(mindis)
                            break
                numberset.append(num)
                distset.append(distanceA)
                temperatureset.append(temperature)
                temperature = temperature*decreaseFactor
          
        return outputList




    

Use the cell below for all python code needed to test the `LondonRailwayMapper` class above.

In [9]:
import timeit
test = LondonRailwayMapper()
starttime = timeit.default_timer()
test.loadStationsAndLines()
endtime = timeit.default_timer()
print("\nExecution time to load:", round(endtime-starttime,10))
fromList = ["Baker Street", "Epping", "Canonbury", "Vauxhall"]
toList = ["North Wembley", "Belsize Park", "Balham", "Leytonstone"]
for i in range(len(fromList)):
    print("From", fromList[i], "to", toList[i])
    minS = test.minStops(fromList[i], toList[i])
    #minD = test.minDistance(ts1[i], ts2[i])
    print("minStops:\t", minS)
    #print("minDistance:\t", minD)
    print("\n")
# ADD YOUR TEST CODE HERE
stationsList = ["Abbey Road", "Barbican", "Bethnal Green", "Cambridge Heath", "Covent Garden", "Dollis Hill", "East Finchley", "Finchley Road and Frognal", "Great Portland Street", "Hackney Wick", "Isleworth", "Kentish Town West", "Leyton", "Marble Arch", "North Wembley", "Old Street", "Pimlico", "Queens Park", "Richmond", "Shepherds Bush", "Tottenham Hale", "Uxbridge", "Vauxhall", "Wapping"]

starttime = timeit.default_timer()
newLine = test.newRailwayLine(stationsList)
endtime = timeit.default_timer()

print("\n\nStation list", stationsList)
print("New station line", newLine)
#print("Total track length from", newLine[0], "to", newLine[len(newLine)-1], ":", testMapper.minDistance(newLine[0], newLine[len(newLine)-1]), "miles")
print("Execution time newLine:", round(endtime-starttime,3))


mindistset = []
numcountset = []
mindistnum = 0
max = 0
min = 113123123

for i in range(10):
    current = float(test.newRailwayLine(stationsList)[len(stationsList)])
    mindistnum += current
    if current > max:
        max = current
    if current < min:
        min = current
    mindistset.append(current)
    numcountset.append(i)

mindistnum = mindistnum/10
print("max:",max)
print("min:",min)
print("average",mindistnum)
print("all in km")
mindistset.sort()






Execution time to load: 0.0065888
From Baker Street to North Wembley
minStops:	 6


From Epping to Belsize Park
minStops:	 17


From Canonbury to Balham
minStops:	 10


From Vauxhall to Leytonstone
minStops:	 6




Station list ['Abbey Road', 'Barbican', 'Bethnal Green', 'Cambridge Heath', 'Covent Garden', 'Dollis Hill', 'East Finchley', 'Finchley Road and Frognal', 'Great Portland Street', 'Hackney Wick', 'Isleworth', 'Kentish Town West', 'Leyton', 'Marble Arch', 'North Wembley', 'Old Street', 'Pimlico', 'Queens Park', 'Richmond', 'Shepherds Bush', 'Tottenham Hale', 'Uxbridge', 'Vauxhall', 'Wapping']
New station line ['Uxbridge', 'North Wembley', 'Dollis Hill', 'East Finchley', 'Tottenham Hale', 'Kentish Town West', 'Finchley Road and Frognal', 'Queens Park', 'Great Portland Street', 'Marble Arch', 'Covent Garden', 'Pimlico', 'Vauxhall', 'Richmond', 'Isleworth', 'Shepherds Bush', 'Wapping', 'Abbey Road', 'Leyton', 'Hackney Wick', 'Cambridge Heath', 'Bethnal Green', 'Old Street', 'Bar

The cell below exemplifies the test code I will invoke on your submission. Do NOT modify it. 

In [10]:
# DO NOT MODIFY THIS CELL

import timeit

testMapper = LondonRailwayMapper()

#
# testing the loadStationsAndLines() API 
#
starttime = timeit.default_timer()
testMapper.loadStationsAndLines()
endtime = timeit.default_timer()
print("\nExecution time to load:", round(endtime-starttime,3))

#
# testing the minStops() and minStops() API on a sample of from/to station pairs  
#
fromList = ["Baker Street", "Epping", "Canonbury", "Vauxhall"]
toList = ["North Wembley", "Belsize Park", "Balham", "Leytonstone"]

for i in range(len(fromList)):
    starttime = timeit.default_timer()
    stops = testMapper.minStops(fromList[i], toList[i])
    endtime = timeit.default_timer()
    print("\nExecution time minStops:", round(endtime-starttime,3))

    starttime = timeit.default_timer()
    dist = testMapper.minStops(fromList[i], toList[i])
    endtime = timeit.default_timer()
    print("Execution time minDistance:", round(endtime-starttime,3))

    print("From", fromList[i], "to", toList[i], "in", stops, "stops and", dist, "miles")  
    
#
# testing the newRailwayLine() API on a small list of stations  
#
stationsList = ["Queens Park", "Chigwell", "Moorgate", "Swiss Cottage", "Liverpool Street", "Highgate"]

starttime = timeit.default_timer()
newLine = testMapper.newRailwayLine(stationsList)
endtime = timeit.default_timer()

print("\n\nStation list", stationsList)
print("New station line", newLine)
print("Total track length from", newLine[0], "to", newLine[len(newLine)-1], ":", testMapper.minDistance(newLine[0], newLine[len(newLine)-1]), "miles")
print("Execution time newLine:", round(endtime-starttime,3))

#
# testing the newRailwayLine() API on a big list of stations  
#
stationsList = ["Abbey Road", "Barbican", "Bethnal Green", "Cambridge Heath", "Covent Garden", "Dollis Hill", "East Finchley", "Finchley Road and Frognal", "Great Portland Street", "Hackney Wick", "Isleworth", "Kentish Town West", "Leyton", "Marble Arch", "North Wembley", "Old Street", "Pimlico", "Queens Park", "Richmond", "Shepherds Bush", "Tottenham Hale", "Uxbridge", "Vauxhall", "Wapping"]

starttime = timeit.default_timer()
newLine = testMapper.newRailwayLine(stationsList)
endtime = timeit.default_timer()

print("\n\nStation list", stationsList)
print("New station line", newLine)
print("Total track length from", newLine[0], "to", newLine[len(newLine)-1], ":", testMapper.minDistance(newLine[0], newLine[len(newLine)-1]), "miles")
print("Execution time newLine:", round(endtime-starttime,3))


Execution time to load: 0.007

Execution time minStops: 0.001
Execution time minDistance: 0.001
From Baker Street to North Wembley in 6 stops and 6 miles

Execution time minStops: 0.003
Execution time minDistance: 0.003
From Epping to Belsize Park in 17 stops and 17 miles

Execution time minStops: 0.007
Execution time minDistance: 0.006
From Canonbury to Balham in 10 stops and 10 miles

Execution time minStops: 0.001
Execution time minDistance: 0.001
From Vauxhall to Leytonstone in 6 stops and 6 miles


Station list ['Queens Park', 'Chigwell', 'Moorgate', 'Swiss Cottage', 'Liverpool Street', 'Highgate']
New station line ['Liverpool Street', 'Moorgate', 'Queens Park', 'Swiss Cottage', 'Highgate', 'Chigwell', 17.205731303776602]


AttributeError: 'NoneType' object has no attribute 'latitude'